In [1]:
!unzip /content/Potato.zip -d /content/Potato

Archive:  /content/Potato.zip
   creating: /content/Potato/Potato/
   creating: /content/Potato/Potato/Potato___Early_blight/
  inflating: /content/Potato/__MACOSX/Potato/._Potato___Early_blight  
  inflating: /content/Potato/Potato/.DS_Store  
  inflating: /content/Potato/__MACOSX/Potato/._.DS_Store  
   creating: /content/Potato/Potato/Potato___Late_blight/
  inflating: /content/Potato/__MACOSX/Potato/._Potato___Late_blight  
   creating: /content/Potato/Potato/Potato___healthy/
  inflating: /content/Potato/__MACOSX/Potato/._Potato___healthy  
  inflating: /content/Potato/Potato/Potato___Early_blight/acb84d77-5b1c-4beb-be27-99fee467bf1f___RS_Early.B 7577.JPG  
  inflating: /content/Potato/__MACOSX/Potato/Potato___Early_blight/._acb84d77-5b1c-4beb-be27-99fee467bf1f___RS_Early.B 7577.JPG  
  inflating: /content/Potato/Potato/Potato___Early_blight/bc7552e4-3901-41b8-8bef-727eeca4270e___RS_Early.B 7942.JPG  
  inflating: /content/Potato/__MACOSX/Potato/Potato___Early_blight/._bc7552e4-39

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the batch size and image dimensions
batch_size = 32
img_height = 224
img_width = 224

# Create an ImageDataGenerator to augment the images and split them into training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

# Load the training data from the Potato directory
train_data = train_datagen.flow_from_directory(
    '/content/Potato/Potato',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

# Load the validation data from the Potato directory
val_data = train_datagen.flow_from_directory(
    '/content/Potato/Potato',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Create a model using a pre-trained ResNet50V2 network
base_model = keras.applications.ResNet50V2(weights='imagenet',
                                           include_top=False,
                                           input_shape=(img_height, img_width, 3))

# Freeze the weights of the pre-trained layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Add a new classification layer to the top of the pre-trained network
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(train_data.num_classes, activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model with an optimizer, loss function, and evaluation metric
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on the training data and validate it on the validation data
model.fit(train_data,
          validation_data=val_data,
          epochs=10,
          steps_per_epoch=train_data.samples // batch_size,
          validation_steps=val_data.samples // batch_size)

# Save the trained model
model.save('potato_image_classifier.h5')


Found 1722 images belonging to 3 classes.
Found 430 images belonging to 3 classes.
Epoch 1/10
53/53 [==============================] - 290s 5s/step - loss: 0.5373 - accuracy: 0.8574 - val_loss: 0.2239 - val_accuracy: 0.9207
Epoch 2/10
53/53 [==============================] - 283s 5s/step - loss: 0.1559 - accuracy: 0.9485 - val_loss: 0.1513 - val_accuracy: 0.9399
Epoch 3/10
53/53 [==============================] - 287s 5s/step - loss: 0.1116 - accuracy: 0.9586 - val_loss: 0.1173 - val_accuracy: 0.9519
Epoch 4/10
53/53 [==============================] - 283s 5s/step - loss: 0.0968 - accuracy: 0.9645 - val_loss: 0.1051 - val_accuracy: 0.9471
Epoch 5/10
53/53 [==============================] - 250s 5s/step - loss: 0.0689 - accuracy: 0.9775 - val_loss: 0.1167 - val_accuracy: 0.9591
Epoch 6/10
53/53 [==============================] - 283s 5s/step - loss: 0.0853 - accuracy: 0.9639 - val_loss: 0.1506 - val_accuracy: 0.9519
Epoch 7/10
53/53 [==============================] - 284s 5s/step - loss

In [15]:
# Get the mapping of class ids to class names
class_mapping = train_data.class_indices

# Print the mapping
print(class_mapping)


{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}


In [10]:
!unzip /content/potato_test.zip -d /content/potato_testing_images

Archive:  /content/potato_test.zip
   creating: /content/potato_testing_images/potato_test/
  inflating: /content/potato_testing_images/potato_test/test_potato_late_blight.jpeg  
  inflating: /content/potato_testing_images/__MACOSX/potato_test/._test_potato_late_blight.jpeg  
  inflating: /content/potato_testing_images/potato_test/test_potato_healthy.jpeg  
  inflating: /content/potato_testing_images/__MACOSX/potato_test/._test_potato_healthy.jpeg  
  inflating: /content/potato_testing_images/potato_test/test_potato_early_blight.jpg  
  inflating: /content/potato_testing_images/__MACOSX/potato_test/._test_potato_early_blight.jpg  


In [18]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image
img_path = '/content/potato_testing_images/potato_test/test_potato_early_blight.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Use the trained model to predict the class of the image
prediction = model.predict(img_array)
class_id = np.argmax(prediction)

# Print the predicted class label
if class_id == 0:
    print('The image is classified as potato_early_blight')
elif class_id == 2:
    print('The image is classified as potato_healthy')
elif class_id == 1:
    print('The image is classified as potato_late_blight')
else:
    print('Unknown class')


1/1 [==============================] - 0s 132ms/step
The image is classified as potato_early_blight


In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image
img_path = '/content/potato_testing_images/potato_test/test_potato_healthy.jpeg'
img = image.load_img(img_path, target_size=(img_height, img_width))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Use the trained model to predict the class of the image
prediction = model.predict(img_array)
class_id = np.argmax(prediction)

# Print the predicted class label
if class_id == 0:
    print('The image is classified as potato_early_blight')
elif class_id == 2:
    print('The image is classified as potato_healthy')
elif class_id == 1:
    print('The image is classified as potato_late_blight')
else:
    print('Unknown class')

1/1 [==============================] - 0s 135ms/step
The image is classified as potato_healthy


In [16]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image
img_path = '/content/potato_testing_images/potato_test/test_potato_late_blight.jpeg'
img = image.load_img(img_path, target_size=(img_height, img_width))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Use the trained model to predict the class of the image
prediction = model.predict(img_array)
class_id = np.argmax(prediction)

# Print the predicted class label
if class_id == 0:
    print('The image is classified as potato_early_blight')
elif class_id == 2:
    print('The image is classified as potato_healthy')
elif class_id == 1:
    print('The image is classified as potato_late_blight')
else:
    print('Unknown class')

1/1 [==============================] - 0s 206ms/step
The image is classified as potato_late_blight
